In [6]:
import scrapy
from scrapy.item import Item, Field
from scrapy.loader import ItemLoader
from scrapy.crawler import CrawlerProcess

In [7]:
class ArticleItem(Item):
    date = Field()
    url = Field()
    title = Field()

In [2]:
class ArticleSpider(scrapy.Spider):
    name = 'article'
    
    def start_requests(self):
        urls = ['https://www.nytimes.com/section/world',]
        
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
        
    
    def parse(self, response):
        load = ItemLoader(item=ArticleItem(), response=response)
        load.add_xpath('date', '//div[@class="product_name"]')
        load.add_xpath('url', '//div[@class="product_title"]')
        load.add_xpath('title', '//h2/self::a')
        
        return load.load_item()

In [3]:
process = CrawlerProcess()
process.crawl(ArticleSpider)
process.start()

2023-01-05 12:20:48 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2023-01-05 12:20:48 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1s  1 Nov 2022), cryptography 38.0.1, Platform Linux-5.10.102.1-microsoft-standard-WSL2-x86_64-with-glibc2.31
2023-01-05 12:20:48 [scrapy.crawler] INFO: Overridden settings:
{}
2023-01-05 12:20:48 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2023-01-05 12:20:48 [scrapy.extensions.telnet] INFO: Telnet Password: a1b94429991d3ae7
2023-01-05 12:20:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-01-05 12:20:48 [scrapy.middleware] INFO: Enabled downloader middlewares:
['s

b'<!DOCTYPE html>\n<html lang="en"  xmlns:og="http://opengraphprotocol.org/schema/">\n  <head>\n    <meta charset="utf-8" />\n    <title data-rh="true">World News - The New York Times</title>\n    <meta data-rh="true" property="og:description" content="The latest international news, investigations and analysis from Africa, the Americas, Asia, Australia, Canada, Europe, the Middle East and the U.K."/><meta data-rh="true" name="description" content="The latest international news, investigations and analysis from Africa, the Americas, Asia, Australia, Canada, Europe, the Middle East and the U.K."/><meta data-rh="true" property="twitter:description" name="description" content="The latest international news, investigations and analysis from Africa, the Americas, Asia, Australia, Canada, Europe, the Middle East and the U.K."/><meta data-rh="true" property="og:title" content="World News"/><meta data-rh="true" property="twitter:title" content="World News"/><meta data-rh="true" property="og:ima